## Get a simple data set object together

In [1]:
import tensorflow as tf
from keras.layers import Conv2D, LeakyReLU, Input, Dense, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
import keras

In [2]:
#Make the dataset
from cycleganstyletransfer.config import DATA_DIR
data_dir = DATA_DIR / "raw"


my_ds_train, my_ds_val = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="both",
    seed=42,
    image_size=(128, 128),
    #sbatch_size = 1,
)

print(my_ds_train.class_names)
print(my_ds_val.class_names)
#print("Class names:", my_ds_train.class_names)
#print("Class indices:", {name: i for i, name in enumerate(my_ds_train.class_names)})

2025-06-06 16:24:08.022 | INFO     | cycleganstyletransfer.config:<module>:11 - PROJ_ROOT path is: C:\Users\willi\Desktop\AIPortfolio\CycleGanV2\cycleganstyletransfer


Found 8230 files belonging to 2 classes.
Using 6584 files for training.
Using 1646 files for validation.
['Images', 'Monet']
['Images', 'Monet']


## Put the model together

In [2]:
def build_discriminator(input_shape=(128, 128, 3)):
    inputs = Input(shape=input_shape, batch_size=32)
    
    # First layer doesn't use instance normalization
    x = Conv2D(32, 3,padding='same')(inputs)
    x = LeakyReLU(0.2)(x)
    x = MaxPooling2D(2)(x)

    x = Conv2D(64, 3,padding='same')(x)
    x = LeakyReLU(0.2)(x)
    x = MaxPooling2D(2)(x)

    x = Conv2D(128, 3,padding='same')(x)
    x = LeakyReLU(0.2)(x)
    x = MaxPooling2D(2)(x)

    x = Conv2D(1, 3, padding='same')(x)

    #x = Flatten()(x)
    #x = GlobalAveragePooling2D()(x)
    #x = Dense(1, activation='sigmoid')(x)
    
    return Model(inputs, x, name='discriminator')

my_discriminator = build_discriminator()
my_discriminator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(32, 128, 128, 3)]       0         
                                                                 
 conv2d (Conv2D)             (32, 128, 128, 32)        896       
                                                                 
 leaky_re_lu (LeakyReLU)     (32, 128, 128, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (32, 64, 64, 32)          0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (32, 64, 64, 64)          18496     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (32, 64, 64, 64)          0         
                                                     

In [4]:
def my_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_true = tf.reshape(y_true, (-1, 1, 1, 1))#Keep whatever batch number
    y_true = tf.broadcast_to(y_true, tf.shape(y_pred))

    return keras.losses.binary_crossentropy(y_true, y_pred, from_logits=True)


In [5]:
from keras.optimizers import Adam

my_discriminator.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss=my_loss, metrics=['accuracy'])
my_discriminator.fit(my_ds_train, validation_data= my_ds_val, epochs=3)

Epoch 1/3
206/206 [==============================] - 76s 364ms/step - loss: 0.7732 - accuracy: 0.8050 - val_loss: 0.5111 - val_accuracy: 0.7675
Epoch 2/3
206/206 [==============================] - 71s 343ms/step - loss: 0.4658 - accuracy: 0.8328 - val_loss: 0.4396 - val_accuracy: 0.8001
Epoch 3/3
206/206 [==============================] - 85s 412ms/step - loss: 0.4255 - accuracy: 0.8396 - val_loss: 0.5662 - val_accuracy: 0.8456


## L2 LOSS AND BATCHSIZE 1 (TO REPLICATE CO DOMAIN CLASSIFIER)

In [3]:
#Make the dataset
from cycleganstyletransfer.config import DATA_DIR
data_dir = DATA_DIR / "raw"


my_ds_train, my_ds_val = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="both",
    seed=42,
    image_size=(128, 128),
    batch_size = 1,
)

my_discriminator = build_discriminator()

2025-06-06 19:22:53.461 | INFO     | cycleganstyletransfer.config:<module>:11 - PROJ_ROOT path is: C:\Users\willi\Desktop\AIPortfolio\CycleGanV2\cycleganstyletransfer


Found 8230 files belonging to 2 classes.
Using 6584 files for training.
Using 1646 files for validation.


In [4]:
def my_square_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_true = tf.reshape(y_true, (-1, 1, 1, 1))#Keep whatever batch number
    y_true = tf.broadcast_to(y_true, tf.shape(y_pred))

    loss = tf.reduce_mean(tf.math.squared_difference(y_true, y_pred))
    return loss#keras.losses.binary_crossentropy(y_true, y_pred, from_logits=True)


In [5]:
from keras.optimizers import Adam

my_discriminator.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss=my_square_loss, metrics=['accuracy'])
my_discriminator.fit(my_ds_train, validation_data= my_ds_val, epochs=3)

Epoch 1/3


6584/6584 [==============================] - 235s 36ms/step - loss: 1.4503 - accuracy: 0.7126 - val_loss: 0.1817 - val_accuracy: 0.8365
Epoch 2/3
6584/6584 [==============================] - 278s 42ms/step - loss: 0.1391 - accuracy: 0.8380 - val_loss: 0.1380 - val_accuracy: 0.8331
Epoch 3/3
6584/6584 [==============================] - 237s 36ms/step - loss: 0.1238 - accuracy: 0.8517 - val_loss: 0.1187 - val_accuracy: 0.8473


In [12]:
my_descrim = build_discriminator()

(None, 256, 256, 64)
(None, 128, 128, 128)
(None, 64, 64, 256)
(None, 32, 32, 512)
(None, 32, 32, 1)


In [15]:
m_photo, i_photo = my_data.get_new()

test_output = my_descrim(m_photo)
print(test_output.shape)
print(test_output)

(1, 1)
tf.Tensor([[0.97778535]], shape=(1, 1), dtype=float32)


## Get a simple training loop together

In [22]:
from keras.losses import BinaryCrossentropy

my_loss = BinaryCrossentropy(from_logits=False)

def discrim_loss(monet_image_output, photo_image_output):

    total_loss = 0.5 * (my_loss(monet_image_output, tf.ones_like(monet_image_output)) + my_loss(photo_image_output, tf.zeros_like(photo_image_output)))
    return total_loss   



In [23]:
from keras.optimizers import Adam

my_optimizer = Adam()


In [26]:
EPOCHS = 1
EPOCH_LENGTH = 10

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    for iteration in range(EPOCH_LENGTH):
        m_photo, i_photo = my_data.get_new()

        with tf.GradientTape() as tape:
            monet_output = my_descrim(m_photo)
            photo_output = my_descrim(i_photo)
            print(monet_output)
            print(photo_output)

            loss = discrim_loss(monet_output, photo_output)

        grads = tape.gradient(loss, my_descrim.trainable_variables)
        my_optimizer.apply_gradients(zip(grads, my_descrim.trainable_variables))
    
        # Calculate accuracy
        monet_accuracy = tf.reduce_mean(tf.cast(monet_output > 0.5, tf.float32))
        photo_accuracy = tf.reduce_mean(tf.cast(photo_output < 0.5, tf.float32))
        total_accuracy = 0.5 * (monet_accuracy + photo_accuracy)

        print(f"Iteration {iteration+1}/{EPOCH_LENGTH}")
        print(f"Loss: {loss:.4f}")
        print(f"Accuracy on Monet images: {monet_accuracy:.2%}")
        print(f"Accuracy on Photo images: {photo_accuracy:.2%}")
        print(f"Total accuracy: {total_accuracy:.2%}\n")

#print(my_descrim.summary())

Epoch 1/1
tf.Tensor([[1.]], shape=(1, 1), dtype=float32)
tf.Tensor([[1.]], shape=(1, 1), dtype=float32)
Iteration 1/10
Loss: 7.7125
Accuracy on Monet images: 100.00%
Accuracy on Photo images: 0.00%
Total accuracy: 50.00%

tf.Tensor([[1.]], shape=(1, 1), dtype=float32)
tf.Tensor([[1.]], shape=(1, 1), dtype=float32)
Iteration 2/10
Loss: 7.7125
Accuracy on Monet images: 100.00%
Accuracy on Photo images: 0.00%
Total accuracy: 50.00%

tf.Tensor([[1.]], shape=(1, 1), dtype=float32)
tf.Tensor([[1.]], shape=(1, 1), dtype=float32)
Iteration 3/10
Loss: 7.7125
Accuracy on Monet images: 100.00%
Accuracy on Photo images: 0.00%
Total accuracy: 50.00%

tf.Tensor([[1.]], shape=(1, 1), dtype=float32)
tf.Tensor([[1.]], shape=(1, 1), dtype=float32)
Iteration 4/10
Loss: 7.7125
Accuracy on Monet images: 100.00%
Accuracy on Photo images: 0.00%
Total accuracy: 50.00%

tf.Tensor([[1.]], shape=(1, 1), dtype=float32)
tf.Tensor([[1.]], shape=(1, 1), dtype=float32)
Iteration 5/10
Loss: 7.7125
Accuracy on Monet im